In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer
import cv2
import numpy as np
import os
import pickle


# ========================================
# 啟動 FaceNet
# ========================================
embedder = FaceNet()
l2_normalizer = Normalizer("l2")

# ========================================
# 臉部偵測器（Haar）
# ========================================
face_detector = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# ========================================
# 建立資料增強器（針對臉部）
# ========================================
datagen = ImageDataGenerator(
    rotation_range=30,         # 左右旋轉
    width_shift_range=0.15,    # 左右平移
    height_shift_range=0.15,   # 上下平移
    shear_range=10,            # 剪切角度
    zoom_range=0.2,            # 縮放
    horizontal_flip=True,      # 水平翻轉
    fill_mode='nearest'
)

# ========================================
# 資料集路徑
# ========================================
DATASET_DIR = "dataset"  # 你的資料夾名稱，例如 dataset/person1/*.jpg
face_db = {}


# ========================================
# 將臉部影像套用資料增強並轉成 embedding
# ========================================
def augment_and_embed(face_img):
    """輸入臉部影像，輸出多個增強後的 FaceNet embeddings"""

    # 調整成 FaceNet 標準輸入（160×160）
    face = cv2.resize(face_img, (160, 160))
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

    # Keras 的增強需要 shape 為 (1, H, W, 3)
    x = face.reshape((1,) + face.shape)

    embeddings = []

    # 生成 5 張增強影像（可以增加數量）
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        augmented = batch[0].astype("uint8")

        # 轉換成 FaceNet embedding
        embedding = embedder.embeddings([augmented])[0]
        embedding = l2_normalizer.transform(embedding.reshape(1, -1))[0]
        embeddings.append(embedding)

        i += 1
        if i >= 5:   # 每張臉額外產生 5 張變形影像
            break

    return embeddings


# ========================================
# 開始建立資料庫（含增強）
# ========================================
for person in os.listdir(DATASET_DIR):
    person_dir = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(person_dir):
        continue

    print(f"📌 正在處理：{person}")
    all_embeddings = []

    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            continue

        x, y, w, h = faces[0]
        face_img = img[y:y+h, x:x+w]

        # 原始影像 embedding
        face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        original_embedding = embedder.embeddings([face_rgb])[0]
        original_embedding = l2_normalizer.transform(original_embedding.reshape(1, -1))[0]
        all_embeddings.append(original_embedding)

        # 加入增強過後的 embedding
        augmented_embeddings = augment_and_embed(face_img)
        all_embeddings.extend(augmented_embeddings)

    face_db[person] = all_embeddings


# ========================================
# 存成 pkl
# ========================================
with open("face_db.pkl", "wb") as f:
    pickle.dump(face_db, f)

print("✅ 已完成資料擴充 + 人臉資料庫更新！")


📌 正在處理：s1254001
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
📌 正在處理：s1254006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
📌 正在處理：s1254019
📌 正在處理：s1254029
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
📌 正在

---

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer
import cv2
import numpy as np
import os
import pickle


# ========================================
# 啟動 FaceNet
# ========================================
embedder = FaceNet()
l2_normalizer = Normalizer("l2")

# ========================================
# 臉部偵測器（Haar）
# ========================================
face_detector = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# ========================================
# 建立資料增強器
# ========================================
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ========================================
# 資料集路徑
# ========================================
DATASET_DIR = "dataset"
face_db = {}


# ========================================
# 顯示資料擴充圖像 + 回傳 FaceNet embeddings
# ========================================
def augment_and_embed(face_img, name):
    """輸入臉部影像 → 顯示增強後的圖片 → 產生 embeddings"""

    # FaceNet 輸入大小 (160x160)
    face = cv2.resize(face_img, (160, 160))
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

    x = face.reshape((1,) + face.shape)

    embeddings = []

    print(f"🔄 正在為 {name} 進行資料擴充...")

    # 產生 5 張增強影像
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        augmented = batch[0].astype("uint8")

        # ===============================
        # 顯示擴充後的圖片
        # ===============================
        show_img = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
        cv2.imshow(f"Augmented Images - {name}", show_img)
        cv2.waitKey(300)   # 每張顯示 300ms

        # 產生 embedding
        embedding = embedder.embeddings([augmented])[0]
        embedding = l2_normalizer.transform(embedding.reshape(1, -1))[0]
        embeddings.append(embedding)

        i += 1
        if i >= 5:
            break

    cv2.destroyAllWindows()
    return embeddings


# ========================================
# 建立資料庫（含資料增強 + 顯示）
# ========================================
for person in os.listdir(DATASET_DIR):
    person_dir = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(person_dir):
        continue

    print(f"📌 正在處理：{person}")
    all_embeddings = []

    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            continue

        x, y, w, h = faces[0]
        face_img = img[y:y+h, x:x+w]

        # 原始 embedding
        face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        original_embedding = embedder.embeddings([face_rgb])[0]
        original_embedding = l2_normalizer.transform(original_embedding.reshape(1, -1))[0]
        all_embeddings.append(original_embedding)

        # 加入資料增強後的 embeddings（同時顯示）
        augmented_embeddings = augment_and_embed(face_img, person)
        all_embeddings.extend(augmented_embeddings)

    face_db[person] = all_embeddings


# ========================================
# 儲存 face_db.pkl
# ========================================
with open("face_db.pkl", "wb") as f:
    pickle.dump(face_db, f)

print("✅ 已完成資料擴充 + 顯示 + 建立 face_db.pkl！")


📌 正在處理：s1254001
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
🔄 正在為 s1254001 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
🔄 正在為 s1254001 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
📌 正在處理：s1254006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
🔄 正在為 s1254006 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
📌 正在處理：s1254019
📌 正在處理：s1254029
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
🔄 正在為 s1254029 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92